In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import EaseOfMovementIndicator
#from ta.volume import SMAEaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.volume import AccDistIndexIndicator
from ta.trend import SMAIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
from ta.others import DailyLogReturnIndicator
from ta.trend import MACD
from ta.trend import EMAIndicator
#from scipy.stats import percentileofscore
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
sip=sip[sip!='BRK.B']
sip=sip[sip!='BF.B']
df=pd.DataFrame()
dfout=pd.DataFrame()
start_date="2000-10-01"
end_date="2021-10-01"
txt = "{:.2%}"
wind=14
for i in range(0,100):
    try:
        stk=sip[i]
        #df = yf.download(stk, start_date, end_date,interval=intr).dropna()
        #print(stk)
        dfd = yf.download(stk, start_date, end_date,interval='1d').dropna()
        dfd['Date'] = pd.to_datetime(dfd.index)
        dfd['Week_Number'] = dfd['Date'].dt.isocalendar().week
        dfd['Year'] = dfd['Date'].dt.isocalendar().year
        dfd['Close1']=dfd['Close'].shift(1)
        dfd['Volume1']=dfd['Volume'].shift(1)
        dfd['Low1']=dfd['Low'].shift(1)
        dfd['High1']=dfd['High'].shift(1)
        dfd['ChangePre']=dfd.Close-dfd.Close1
        dfd['Change']=dfd.Close-dfd.Open
        dfd['ChangePreP']=dfd.ChangePre/dfd.Close1
        dfd['ChangeP']=dfd.Change/dfd.Open
        dfd.set_index(['Year','Week_Number'],inplace=True)
        dfd['Mon']=dfd['Date'].dt.strftime('%m-%Y')


        dfw = dfd.groupby(['Year','Week_Number']).agg({'Date':'first','Open':'first', 'High':'max', 'Low':'min', 'Close':'last','Volume':'sum'})
        dfw['Close1']=dfw['Close'].shift(1)
        dfw['Volume1']=dfw['Volume'].shift(1)
        dfw['Low1']=dfw['Low'].shift(1)
        dfw['High1']=dfw['High'].shift(1)

        dfd=pd.merge(left=dfd,right=dfw,how='left',left_index=True,right_index=True,suffixes=[None,'w'])
        #print(dfd.tail)
        dfm=dfd.resample('M', on='Date').agg({'Date':'first','Open':'first', 'High':'max', 'Low':'min', 'Close':'last','Volume':'sum','Mon':'first'})
        dfm['Close1']=dfm['Close'].shift(1)
        dfm['Volume1']=dfm['Volume'].shift(1)
        dfm['Low1']=dfm['Low'].shift(1)
        dfm['High1']=dfm['High'].shift(1)

        dfd=pd.merge(left=dfd,right=dfm,how='left',left_on='Mon',right_on='Mon',suffixes=[None,'m'])
        dfd=dfd.dropna()

        
        #dfd['fid']=ForceIndexIndicator(close=dfd.Close1,volume=dfd.Volume1).force_index()
        #dfd['fiw']=ForceIndexIndicator(close=dfd.Close1w,volume=dfd.Volume1w).force_index()
        #dfd['fim']=ForceIndexIndicator(close=dfd.Close1m,volume=dfd.Volume1m).force_index()
        #dfd['emd']=EaseOfMovementIndicator(high=dfd.High1,low=dfd.Low1,volume=dfd.Volume1).ease_of_movement()
        #dfd['emw']=EaseOfMovementIndicator(high=dfd.High1w,low=dfd.Low1w,volume=dfd.Volume1w).ease_of_movement()
        dfd['em_m']=EaseOfMovementIndicator(high=dfd.High1m,low=dfd.Low1m,volume=dfd.Volume1m).ease_of_movement()
        #dfd['semd']=EaseOfMovementIndicator(high=dfd.High1,low=dfd.Low1,volume=dfd.Volume1).sma_ease_of_movement()
        dfd['sem_w']=EaseOfMovementIndicator(high=dfd.High1w,low=dfd.Low1w,volume=dfd.Volume1w).sma_ease_of_movement()
        dfd['sem_m']=EaseOfMovementIndicator(high=dfd.High1m,low=dfd.Low1m,volume=dfd.Volume1m).sma_ease_of_movement()
       # dfd['mac_d']=MACD(close=dfd.Close1).macd_diff()
        dfd['mac_w']=MACD(close=dfd.Close1w).macd_diff()
       # dfd['mac_m']=MACD(close=dfd.Close1m).macd_diff()
        dfd['dlr_d']=DailyLogReturnIndicator(close=dfd.Close1).daily_log_return() 
        dfd['ema_d']=EMAIndicator(close=dfd.Close1,window=14).ema_indicator()
        dfd['ema_w']=EMAIndicator(close=dfd.Close1w,window=14).ema_indicator()
        dfd['ema_m']=EMAIndicator(close=dfd.Close1m,window=14).ema_indicator()
       # dfd['bblid']=BollingerBands(close=dfd.Close1).bollinger_lband_indicator()
        dfd['adi_d']=AccDistIndexIndicator(high=dfd.High1,low=dfd.Low1,volume=dfd.Volume1,close=dfd.Close1).acc_dist_index()
        
        dfd['dcl_w']=DonchianChannel(high=dfd.High1w,low=dfd.Low1w,close=dfd.Close1w).donchian_channel_lband() 
        dfd['dcl_m']=DonchianChannel(high=dfd.High1m,low=dfd.Low1m,close=dfd.Close1m).donchian_channel_lband() 
        
        dfd=dfd.dropna()
        #print(dfd.head)
        x=1
        x2=50
        #x3=50
        #print(dfd.columns)
        conditions = [ 
        (dfd['Open']>0)

        #& (dfd.fid>np.percentile(dfd.fid,100-x))
        #& (dfd.fiw>np.percentile(dfd.fiw,100-x2))
        #& (dfd.fim>np.percentile(dfd.fim,100-x3))
       # & (dfd.mcw>np.percentile(dfd.mcw,100-x))    
        #& (dfd.emd>np.percentile(dfd.emd,100-x))
        #& (dfd.emw>np.percentile(dfd.emw,100-x))
        #& (dfd.emm>np.percentile(dfd.emm,100-x))
        #& (dfd.semw>np.percentile(dfd.semw,100-x))
        #& (dfd.semm>np.percentile(dfd.semm,100-x))
        #&(dfd.dlr_d>np.percentile(dfd.dlr_d,100-x))
            
        &(dfd.adi_d>np.percentile(dfd.adi_d,99))    
            
            
        &(dfd.ema_d>np.percentile(dfd.ema_d,10))
        &(dfd.ema_w>np.percentile(dfd.ema_w,10))
        &(dfd.ema_m>np.percentile(dfd.ema_m,10))
        &(dfd.mac_w>np.percentile(dfd.mac_w,10))      
        &(dfd.sem_w>np.percentile(dfd.sem_w,10))
        ,
        (dfd['Open']>0)
        #& (dfd.fid<np.percentile(dfd.fid,x))
        #& (dfd.fiw<np.percentile(dfd.fiw,x2))
       # & (dfd.fim<np.percentile(dfd.fim,x3))
        #& (dfd.mcw<np.percentile(dfd.mcw,x))
        #& (dfd.emd<np.percentile(dfd.emd,x))
        #& (dfd.emw<np.percentile(dfd.emw,x))
        #& (dfd.emm<np.percentile(dfd.emm,x))
        #& (dfd.semw<np.percentile(dfd.semw,x))
        #& (dfd.semm<np.percentile(dfd.semm,x))
        #&(dfd.dlr_d<np.percentile(dfd.dlr_d,x))
        #&(dfd.ema<np.percentile(dfd.ema,1))
        &(dfd.ema_d<np.percentile(dfd.ema_d,1))
            
        &(dfd.adi_d<np.percentile(dfd.adi_d,90))
        &(dfd.dcl_w<np.percentile(dfd.dcl_w,90))
        &(dfd.dcl_m<np.percentile(dfd.dcl_m,90))
        &(dfd.mac_w<np.percentile(dfd.mac_w,90))      
        &(dfd.sem_w<np.percentile(dfd.sem_w,90))
        ]

        choices = [
        -1,1
        ]
        dfd['trade_pass']=np.select(conditions,choices,default=0)
        dfd_long=dfd[dfd['trade_pass']==1]
        dfd_short=dfd[dfd['trade_pass']==-1]
        fir=dfd.iloc[0,0]
        las=dfd.iloc[len(dfd.index)-1,3]

        hold_prof=(las)/(fir)-1
        hold_wr=len(dfd[dfd.ChangePre>0])/(len(dfd.index))
        hold_avg=np.sum(dfd.ChangePreP)/len(dfd.index)
        hold_tr=1
        hold_drw=np.min(dfd.ChangePreP)
        
        long_prof=(np.sum(dfd_long.Change)+fir)/fir-1
        long_prof_pre=(np.sum(dfd_long.ChangePre)+fir)/fir-1
        long_wr=len(dfd_long[dfd_long.Change>0])/(len(dfd_long.index))
        long_wr_pre=len(dfd_long[dfd_long.ChangePre>0])/(len(dfd_long.index))
        long_ave=np.sum(dfd_long.ChangeP)/len(dfd_long.index)
        long_ave_pre=np.sum(dfd_long.ChangePreP)/len(dfd_long.index)   
        long_tr=len(dfd_long.index)/(len(dfd.index))
        long_drw=np.min(dfd_long.ChangeP)
        long_drw_pre=np.min(dfd_long.ChangePreP)

        short_prof=(-np.sum(dfd_short.Change)+fir)/fir-1
        short_prof_pre=(-np.sum(dfd_short.ChangePre)+fir)/fir-1
        short_wr=len(dfd_short[dfd_short.Change<0])/(len(dfd_short.index))
        short_wr_pre=len(dfd_short[dfd_short.ChangePre<0])/(len(dfd_short.index))
        short_ave=(-np.sum(dfd_short.ChangeP))/len(dfd_short.index)
        short_ave_pre=(-np.sum(dfd_short.ChangePreP))/len(dfd_short.index)   
        short_tr=len(dfd_short.index)/(len(dfd.index))
        short_drw=np.max(dfd_short.ChangeP)
        short_drw_pre=np.max(dfd_short.ChangePreP)



        temp=pd.Series([stk,
                        hold_prof,hold_wr,hold_avg,hold_tr,hold_drw,
                        long_prof,long_wr,long_ave,long_tr,long_drw,
                        long_prof_pre,long_wr_pre,long_ave_pre,long_tr,long_drw_pre,
                        short_prof,short_wr,short_ave,short_tr,short_drw,
                        short_prof_pre,short_wr_pre,short_ave_pre,short_tr,short_drw_pre])
        dfout=dfout.append(temp,ignore_index=True)
    except Exception as e: print(e)

print(dfout)
fin1=np.array([])
fin2=np.array([])
fin3=np.array([])
fin4=np.array([])
fin5=np.array([])
for j in range (1,6):
    temp1=(np.average(dfout.iloc[:,j]))    
    fin1=np.append(fin1,temp1)
for j in range (6,11):
    temp2=(np.average(dfout.iloc[:,j]))    
    fin2=np.append(fin2,temp2)
for j in range (11,16):
    temp3=(np.average(dfout.iloc[:,j]))    
    fin3=np.append(fin3,temp3)
for j in range (16,21):
    temp4=(np.average(dfout.iloc[:,j]))    
    fin4=np.append(fin4,temp4)
for j in range (21,26):
    temp5=(np.average(dfout.iloc[:,j]))    
    fin5=np.append(fin5,temp5)   
    
    
    
    
dff1=pd.DataFrame(fin1.reshape(1,-1))
dff2=pd.DataFrame(fin2.reshape(1,-1))
dff3=pd.DataFrame(fin3.reshape(1,-1))
dff4=pd.DataFrame(fin4.reshape(1,-1))
dff5=pd.DataFrame(fin5.reshape(1,-1))




#print(dff1)
#print(dff2)
dff=pd.concat([dff1,dff2,dff3,dff4,dff5])
dff.columns=(['total%','win_rate%','avg%/trade','trade rate','drawdown'])
dff.index=(['hold','long-day','long-overnight','short-day','short-overnight'])
#dff=pd.DataFrame(fin.reshape(1,-1))
#dff.columns=(['hold_prof','long_prof','long_prof_pre','long_wr','long_wr_pre','long_ave','long_ave_pre','trade_rate_long'])
print(dff)
dff.to_csv('bt.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********